 **Scraping des données**

In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

base_url = "http://www.tunisie-annonce.com/AnnoncesImmobilier.asp"
annonces_data = []
MAX_PAGES = 5

# Fonction pour extraire les détails depuis la page de l'annonce
def extraire_details(detail_url):
    try:
        response = requests.get(detail_url)
        response.encoding = "ISO-8859-1"
        soup = BeautifulSoup(response.text, "html.parser")

        cat, surface, description = "", "", ""
        date_insert, date_modif, localisation = "", "", ""

        # Catégorie
        cat_td = soup.find("td", string="Catégorie")
        if cat_td and cat_td.find_next_sibling("td"):
            cat = cat_td.find_next_sibling("td").get_text(strip=True)

        # Surface
        surface_td = soup.find("td", string="Surface")
        if surface_td and surface_td.find_next_sibling("td"):
            surface = surface_td.find_next_sibling("td").get_text(strip=True)

        # Description
        desc_td = soup.find("td", string="Texte")
        if desc_td and desc_td.find_next_sibling("td"):
            description = desc_td.find_next_sibling("td").get_text(strip=True)

        # Date insérée
        insert_td = soup.find("td", string="Insérée le")
        if insert_td and insert_td.find_next_sibling("td"):
            date_insert = insert_td.find_next_sibling("td").get_text(strip=True)

        # Date modifiée
        modif_td = soup.find("td", string="Modifiée le")
        if modif_td and modif_td.find_next_sibling("td"):
            date_modif = modif_td.find_next_sibling("td").get_text(strip=True)

        # Localisation
        localisation_td = soup.find("td", string="Localisation")
        if localisation_td and localisation_td.find_next_sibling("td"):
            localisation = localisation_td.find_next_sibling("td").get_text(" > ", strip=True)

        return cat, surface, description, date_insert, date_modif, localisation

    except Exception as e:
        print(f"❌ Erreur en accédant à {detail_url}")
        return "", "", "", "", "", ""

# Scraping avec pagination
page = 1
while page <= MAX_PAGES:
    print(f"🔎 Scraping page {page}...")

    params = {"num": page}
    response = requests.get(base_url, params=params)
    response.encoding = "ISO-8859-1"

    if response.status_code != 200:
        print("❌ Erreur de chargement de la page.")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    annonces = soup.find_all("tr", class_="Tableau1")

    if not annonces:
        print("✅ Fin des annonces.")
        break

    for annonce in annonces:
        cols = annonce.find_all("td")
        if len(cols) >= 12:
            region = cols[1].get_text(strip=True)
            nature = cols[3].get_text(strip=True)
            type_annonce = cols[5].get_text(strip=True)
            texte_annonce = cols[7].get_text(strip=True)  # Champ visible
            prix = cols[9].get_text(strip=True)

            # Nettoyage du prix
            prix = re.sub(r"Dinar Tunisien.*", "", prix).replace("\xa0", " ").strip()

            # Lien vers page de détails
            link_tag = cols[7].find("a")
            detail_url = ""
            if link_tag and "href" in link_tag.attrs:
                detail_url = "http://www.tunisie-annonce.com/" + link_tag["href"]

                # Extraire les détails
                cat, surface, desc, date_insert, date_modif, localisation = extraire_details(detail_url)

                # Séparer la localisation : Pays / Ville uniquement
                pays, ville = "", ""
                if localisation:
                    parts = localisation.split(" > ")
                    if len(parts) >= 1:
                        pays = parts[0]
                    if len(parts) >= 3:
                        ville = parts[2]  # On saute gouvernorat

                # Stocker les données
                annonces_data.append({
                    "Lien": detail_url,
                    "Catégorie": cat,
                    "Surface": surface,
                    "Texte complet": desc,
                    "Texte annonce": texte_annonce,
                    "Insérée le": date_insert,
                    "Modifiée le": date_modif,
                    "Pays": pays,
                    "Ville": ville,
                    "Région": region,
                    "Nature": nature,
                    "Type": type_annonce,
                    "Prix (TND)": prix
                })

    page += 1

# Export en Excel
df = pd.DataFrame(annonces_data)
df.to_excel("annoncesv2.xlsx", index=False, engine="openpyxl")
print(f"✅ {len(df)} annonces sauvegardées dans 'annoncesv2.xlsx'")


🔎 Scraping page 1...
🔎 Scraping page 2...
🔎 Scraping page 3...
🔎 Scraping page 4...
🔎 Scraping page 5...
✅ 125 annonces sauvegardées dans 'annoncesv2.xlsx'


**Stockage des données dans SQLite**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sqlite3

base_url = "http://www.tunisie-annonce.com/AnnoncesImmobilier.asp"
MAX_PAGES = 5

# Connexion à SQLite
conn = sqlite3.connect("annonces.db")
cursor = conn.cursor()

# Création de la table sans la colonne 'gouvernorat'
cursor.execute("""
    CREATE TABLE IF NOT EXISTS annonces (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        lien TEXT,
        categorie TEXT,
        surface TEXT,
        texte_complet TEXT,
        texte_annonce TEXT,
        inseree_le TEXT,
        modifiee_le TEXT,
        pays TEXT,
        ville TEXT,
        region TEXT,
        nature TEXT,
        type TEXT,
        prix_tnd TEXT
    )
""")
conn.commit()

def extraire_details(detail_url):
    try:
        response = requests.get(detail_url)
        response.encoding = "ISO-8859-1"
        soup = BeautifulSoup(response.text, "html.parser")

        cat, surface, description = "", "", ""
        date_insert, date_modif, localisation = "", "", ""

        # Catégorie
        cat_td = soup.find("td", string="Catégorie")
        if cat_td and cat_td.find_next_sibling("td"):
            cat = cat_td.find_next_sibling("td").get_text(strip=True)

        # Surface
        surface_td = soup.find("td", string="Surface")
        if surface_td and surface_td.find_next_sibling("td"):
            surface = surface_td.find_next_sibling("td").get_text(strip=True)

        # Description
        desc_td = soup.find("td", string="Texte")
        if desc_td and desc_td.find_next_sibling("td"):
            description = desc_td.find_next_sibling("td").get_text(strip=True)

        # Dates
        insert_td = soup.find("td", string="Insérée le")
        if insert_td and insert_td.find_next_sibling("td"):
            date_insert = insert_td.find_next_sibling("td").get_text(strip=True)

        modif_td = soup.find("td", string="Modifiée le")
        if modif_td and modif_td.find_next_sibling("td"):
            date_modif = modif_td.find_next_sibling("td").get_text(strip=True)

        # Localisation
        localisation_td = soup.find("td", string="Localisation")
        if localisation_td and localisation_td.find_next_sibling("td"):
            localisation = localisation_td.find_next_sibling("td").get_text(" > ", strip=True)

        return cat, surface, description, date_insert, date_modif, localisation

    except Exception as e:
        print(f"❌ Erreur en accédant à {detail_url}")
        return "", "", "", "", "", ""

# Scraping
page = 1
while page <= MAX_PAGES:
    print(f"🔎 Scraping page {page}...")

    params = {"num": page}
    response = requests.get(base_url, params=params)
    response.encoding = "ISO-8859-1"

    if response.status_code != 200:
        print("❌ Erreur de chargement de la page.")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    annonces = soup.find_all("tr", class_="Tableau1")

    if not annonces:
        break

    for annonce in annonces:
        cols = annonce.find_all("td")
        if len(cols) >= 12:
            region = cols[1].get_text(strip=True)
            nature = cols[3].get_text(strip=True)
            type_annonce = cols[5].get_text(strip=True)
            texte_annonce = cols[7].get_text(strip=True)
            prix = cols[9].get_text(strip=True)

            # Nettoyage du prix
            prix = re.sub(r"Dinar Tunisien.*", "", prix).replace("\xa0", " ").strip()

            # Lien vers page de détails
            link_tag = cols[7].find("a")
            if link_tag and "href" in link_tag.attrs:
                detail_url = "http://www.tunisie-annonce.com/" + link_tag["href"]

                cat, surface, desc, date_insert, date_modif, localisation = extraire_details(detail_url)

                # Localisation : on garde seulement pays et ville
                pays, ville = "", ""
                if localisation:
                    parts = localisation.split(" > ")
                    if len(parts) >= 1:
                        pays = parts[0]
                    if len(parts) >= 3:
                        ville = parts[2]

                # Insertion dans SQLite
                cursor.execute("""
                    INSERT INTO annonces (
                        lien, categorie, surface, texte_complet, texte_annonce,
                        inseree_le, modifiee_le, pays, ville,
                        region, nature, type, prix_tnd
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    detail_url, cat, surface, desc, texte_annonce,
                    date_insert, date_modif, pays, ville,
                    region, nature, type_annonce, prix
                ))

    page += 1

# Sauvegarder
conn.commit()
conn.close()

print("✅ Données enregistrées dans la base SQLite 'annonces.db'")


🔎 Scraping page 1...
🔎 Scraping page 2...
🔎 Scraping page 3...
🔎 Scraping page 4...
🔎 Scraping page 5...
✅ Données enregistrées dans la base SQLite 'annonces.db'


In [ ]:
import sqlite3
pd.read_sql("SELECT * FROM annonces", sqlite3.connect("/content/annonces.db"))


id                                               lien  \
0      1  http://www.tunisie-annonce.com/DetailsAnnonceI...   
1      2  http://www.tunisie-annonce.com/DetailsAnnonceI...   
2      3  http://www.tunisie-annonce.com/DetailsAnnonceI...   
3      4  http://www.tunisie-annonce.com/DetailsAnnonceI...   
4      5  http://www.tunisie-annonce.com/DetailsAnnonceI...   
..   ...                                                ...   
120  121  http://www.tunisie-annonce.com/DetailsAnnonceI...   
121  122  http://www.tunisie-annonce.com/DetailsAnnonceI...   
122  123  http://www.tunisie-annonce.com/DetailsAnnonceI...   
123  124  http://www.tunisie-annonce.com/DetailsAnnonceI...   
124  125  http://www.tunisie-annonce.com/DetailsAnnonceI...   

                           categorie   surface  \
0          Offres>Terrain>Terrain nu    340 m²   
1          Offres>Terrain>Terrain nu  1 000 m²   
2          Offres>Terrain>Terrain nu  1 080 m²   
3          Offres>Terrain>Terrain nu    322 m²   
4          Offres>Terrain>Terrain nu    776 m²   
..                               ...       ...   
120          Offres>Location>Maisons      1 m²   
121  Offres>Location>Appart. 1 pièce     70 m²   
122          Offres>Location>Maisons    120 m²   
123          Offres>Location>Maisons     70 m²   
124            Offres>Vente>Surfaces    370 m²   

                                         texte_complet  \
0    ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...   
1    ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...   
2    ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...   
3    je  dix  bien  constructible  aussi  zone  vil...   
4    zone  acceptable  ((   pas  de  zone  populair...   
..                                                 ...   
120  a louer une villa s4 moderne située à la soukr...   
121  un s+1 totalement meublé  a louer a soukra dan...   
122  un luxueux s+2 entièrement meublé  a la soukra...   
123  un s+1 entièrement meublé  a louer a soukra da...   
124  a vendre terrain sis a mnihla clôture  dans un...   

                     texte_annonce  inseree_le modifiee_le     pays    ville  \
0               Terrain nu manouba  10/12/2024  22/03/2025  Tunisie  Manouba   
1              Terrain nu chorfech  20/04/2024  22/03/2025  Tunisie   Ariana   
2                Jabbes terrain nu  18/10/2022  22/03/2025  Tunisie   Ariana   
3    Zone  villas  ter 321m2  cons  22/07/2024  22/03/2025  Tunisie   Ariana   
4        Aux  choix   776m2  507m2  22/07/2024  22/03/2025  Tunisie   Ariana   
..                             ...         ...         ...      ...      ...   
120        Villa s4 avec piscine à  20/03/2025  22/03/2025  Tunisie   Ariana   
121            S1 meublé la soukra  02/04/2021  22/03/2025  Tunisie   Ariana   
122        S2 meublé a soukra 90 d  02/04/2021  22/03/2025  Tunisie   Ariana   
123             S1 meublé a soukra  24/06/2021  22/03/2025  Tunisie   Ariana   
124                 Terrain mnihla  11/02/2025  22/03/2025  Tunisie   Ariana   

          region    nature         type prix_tnd  
0    La Mannouba   Terrain   Terrain nu  221 000  
1       Chorfech   Terrain   Terrain nu  150 000  
2         Jabbes   Terrain   Terrain nu  248 400  
3     Chotrana 1   Terrain   Terrain nu  240 000  
4         Ariana   Terrain   Terrain nu  271 000  
..           ...       ...          ...      ...  
120   Chotrana 3  Location      Maisons    7 000  
121    La Soukra  Location  App. 1 pièc      600  
122    La Soukra  Location      Maisons      600  
123    La Soukra  Location      Maisons      550  
124       Mnihla     Vente     Surfaces  300 000  

[125 rows x 14 columns]

**Développement de l’API REST**

1-Installation des dépendances

In [ ]:
!pip install fastapi uvicorn



In [ ]:
 !pip install nest_asyncio pyngrok

In [ ]:
 !pip install  openpyxl bs4

2-Implémentation de l’API avec FastAPI

In [ ]:
!ngrok config add-authtoken 2ub8U4p67DYslVunez1y39LwG3G_679fw6hYuxbWNQZy3QTbW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


=> API REST opérationnelle permetant d’extraire et d’accéder aux données sur POSTMAN.  

In [ ]:
import requests
from bs4 import BeautifulSoup
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import re

app = FastAPI()
nest_asyncio.apply()

annonces_data = []
MAX_PAGES = 2

def extraire_details(detail_url):
    try:
        response = requests.get(detail_url)
        response.encoding = "ISO-8859-1"
        soup = BeautifulSoup(response.text, "html.parser")

        cat, surface, description = "", "", ""
        date_insert, date_modif, localisation = "", "", ""

        cat_td = soup.find("td", string="Catégorie")
        if cat_td and cat_td.find_next_sibling("td"):
            cat = cat_td.find_next_sibling("td").get_text(strip=True)

        surface_td = soup.find("td", string="Surface")
        if surface_td and surface_td.find_next_sibling("td"):
            surface = surface_td.find_next_sibling("td").get_text(strip=True)

        desc_td = soup.find("td", string="Texte")
        if desc_td and desc_td.find_next_sibling("td"):
            description = desc_td.find_next_sibling("td").get_text(strip=True)

        insert_td = soup.find("td", string="Insérée le")
        if insert_td and insert_td.find_next_sibling("td"):
            date_insert = insert_td.find_next_sibling("td").get_text(strip=True)

        modif_td = soup.find("td", string="Modifiée le")
        if modif_td and modif_td.find_next_sibling("td"):
            date_modif = modif_td.find_next_sibling("td").get_text(strip=True)

        localisation_td = soup.find("td", string="Localisation")
        if localisation_td and localisation_td.find_next_sibling("td"):
            localisation = localisation_td.find_next_sibling("td").get_text(" > ", strip=True)

        return cat, surface, description, date_insert, date_modif, localisation

    except Exception as e:
        print(f"❌ Erreur dans {detail_url}")
        return "", "", "", "", "", ""

# Fonction de scraping
def scraper():
    global annonces_data
    annonces_data.clear()
    base_url = "http://www.tunisie-annonce.com/AnnoncesImmobilier.asp"

    page = 1
    while page <= MAX_PAGES:
        print(f"🔎 Scraping page {page}...")

        params = {"num": page}
        response = requests.get(base_url, params=params)
        response.encoding = "ISO-8859-1"

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, "html.parser")
        annonces = soup.find_all("tr", class_="Tableau1")

        if not annonces:
            break

        for annonce in annonces:
            cols = annonce.find_all("td")
            if len(cols) >= 12:
                region = cols[1].get_text(strip=True)
                nature = cols[3].get_text(strip=True)
                type_annonce = cols[5].get_text(strip=True)
                texte_annonce = cols[7].get_text(strip=True)
                prix = cols[9].get_text(strip=True)

                prix = re.sub(r"Dinar Tunisien.*", "", prix).replace("\xa0", " ").strip()

                link_tag = cols[7].find("a")
                detail_url = ""
                if link_tag and "href" in link_tag.attrs:
                    detail_url = "http://www.tunisie-annonce.com/" + link_tag["href"]

                    cat, surface, desc, date_insert, date_modif, localisation = extraire_details(detail_url)

                    pays, ville = "", ""
                    if localisation:
                        parts = localisation.split(" > ")
                        if len(parts) >= 1:
                            pays = parts[0]
                        if len(parts) >= 3:
                            ville = parts[2]

                    annonces_data.append({
                        "Lien": detail_url,
                        "Catégorie": cat,
                        "Surface": surface,
                        "Texte complet": desc,
                        "Texte annonce": texte_annonce,
                        "Insérée le": date_insert,
                        "Modifiée le": date_modif,
                        "Pays": pays,
                        "Ville": ville,
                        "Région": region,
                        "Nature": nature,
                        "Type": type_annonce,
                        "Prix (TND)": prix
                    })

        page += 1

    return annonces_data

# Endpoint GET
@app.get("/annonces")
def get_annonces():
    return {"total": len(annonces_data), "data": annonces_data}

# Endpoint POST
@app.post("/scrape")
def run_scraper():
    scraper()
    return {"message": "Scraping terminé", "total": len(annonces_data)}

# Ngrok + Serveur
port = 8000
public_url = ngrok.connect(port)
print(f"🚀 API accessible ici : {public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=port)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-28' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

🚀 API accessible ici : NgrokTunnel: "https://f949-35-201-170-126.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [275]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     102.157.126.242:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.157.126.242:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.157.126.242:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.157.126.242:0 - "GET /openapi.json HTTP/1.1" 200 OK
🔎 Scraping page 1...


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

🔎 Scraping page 2...
INFO:     102.157.126.242:0 - "POST /scrape HTTP/1.1" 200 OK
INFO:     102.157.126.242:0 - "GET /annonces HTTP/1.1" 200 OK
🔎 Scraping page 1...
🔎 Scraping page 2...
INFO:     102.157.126.242:0 - "POST /scrape HTTP/1.1" 200 OK
INFO:     102.157.126.242:0 - "GET /annonces HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [275]




---
**PART 2**



 ***Phase 1 : NLP & Enrichissement des données***


Étapes du script :
Chargement de excel

Nettoyage des textes

Résumer les descriptions

Extraction des tags via règles simples

Sauvegarder un nouveau excel enrichi

In [1]:
import pandas as pd
import re
from tqdm import tqdm
from transformers import pipeline
df = pd.read_excel("/content/drive/MyDrive/TP Python/annoncesv2.xlsx")
df.head()

Lien  \
0  http://www.tunisie-annonce.com/DetailsAnnonceI...   
1  http://www.tunisie-annonce.com/DetailsAnnonceI...   
2  http://www.tunisie-annonce.com/DetailsAnnonceI...   
3  http://www.tunisie-annonce.com/DetailsAnnonceI...   
4  http://www.tunisie-annonce.com/DetailsAnnonceI...   

                   Catégorie   Surface  \
0  Offres>Terrain>Terrain nu    340 m²   
1  Offres>Terrain>Terrain nu  1 000 m²   
2  Offres>Terrain>Terrain nu  1 080 m²   
3  Offres>Terrain>Terrain nu    322 m²   
4  Offres>Terrain>Terrain nu    776 m²   

                                       Texte complet  \
0  ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...   
1  ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...   
2  ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...   
3  je  dix  bien  constructible  aussi  zone  vil...   
4  zone  acceptable  ((   pas  de  zone  populair...   

                   Texte annonce  Insérée le Modifiée le     Pays    Ville  \
0             Terrain nu manouba  10/12/2024  22/03/2025  Tunisie  Manouba   
1            Terrain nu chorfech  20/04/2024  22/03/2025  Tunisie   Ariana   
2              Jabbes terrain nu  18/10/2022  22/03/2025  Tunisie   Ariana   
3  Zone  villas  ter 321m2  cons  22/07/2024  22/03/2025  Tunisie   Ariana   
4      Aux  choix   776m2  507m2  22/07/2024  22/03/2025  Tunisie   Ariana   

        Région   Nature        Type Prix (TND)  
0  La Mannouba  Terrain  Terrain nu    221 000  
1     Chorfech  Terrain  Terrain nu    150 000  
2       Jabbes  Terrain  Terrain nu    248 400  
3   Chotrana 1  Terrain  Terrain nu    240 000  
4       Ariana  Terrain  Terrain nu    271 000

In [2]:
#etape 1 : Nettoyage simple des textes
def nettoyer_texte(texte):
    if pd.isna(texte):
        return ""
    texte = str(texte).lower()
    texte = re.sub(r"[^\w\s]", " ", texte)  # remove punctuation
    texte = re.sub(r"\s+", " ", texte)      # remove extra spaces
    return texte.strip()

df["texte_clean"] = df["Texte complet"].apply(nettoyer_texte)



In [3]:
df.head()

Lien  \
0  http://www.tunisie-annonce.com/DetailsAnnonceI...   
1  http://www.tunisie-annonce.com/DetailsAnnonceI...   
2  http://www.tunisie-annonce.com/DetailsAnnonceI...   
3  http://www.tunisie-annonce.com/DetailsAnnonceI...   
4  http://www.tunisie-annonce.com/DetailsAnnonceI...   

                   Catégorie   Surface  \
0  Offres>Terrain>Terrain nu    340 m²   
1  Offres>Terrain>Terrain nu  1 000 m²   
2  Offres>Terrain>Terrain nu  1 080 m²   
3  Offres>Terrain>Terrain nu    322 m²   
4  Offres>Terrain>Terrain nu    776 m²   

                                       Texte complet  \
0  ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...   
1  ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...   
2  ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...   
3  je  dix  bien  constructible  aussi  zone  vil...   
4  zone  acceptable  ((   pas  de  zone  populair...   

                   Texte annonce  Insérée le Modifiée le     Pays    Ville  \
0             Terrain nu manouba  10/12/2024  22/03/2025  Tunisie  Manouba   
1            Terrain nu chorfech  20/04/2024  22/03/2025  Tunisie   Ariana   
2              Jabbes terrain nu  18/10/2022  22/03/2025  Tunisie   Ariana   
3  Zone  villas  ter 321m2  cons  22/07/2024  22/03/2025  Tunisie   Ariana   
4      Aux  choix   776m2  507m2  22/07/2024  22/03/2025  Tunisie   Ariana   

        Région   Nature        Type Prix (TND)  \
0  La Mannouba  Terrain  Terrain nu    221 000   
1     Chorfech  Terrain  Terrain nu    150 000   
2       Jabbes  Terrain  Terrain nu    248 400   
3   Chotrana 1  Terrain  Terrain nu    240 000   
4       Ariana  Terrain  Terrain nu    271 000   

                                         texte_clean  
0  ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...  
1  ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...  
2  ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...  
3  je dix bien constructible aussi zone villas a ...  
4  zone acceptable pas de zone populaire a raoued...

In [4]:
#etape 2 : Résumé automatique
print("Génération des résumés...")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", tokenizer="sshleifer/distilbart-cnn-12-6")
df["resume_annonce"] = df["texte_clean"].apply(lambda x: summarizer(x, max_length=40, min_length=15, do_sample=False)[0]['summary_text'] if x else "")



Génération des résumés...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 détection de langue + résumé par langue
Français ou Anglais → utiliser sshleifer/distilbart-cnn-12-6

Arabe → utiliser un modèle mT5 ou MBart multilingue (facebook/mbart-large-50-many-to-many-mmt)

In [10]:
pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d3af2bebb3ffb6e2547e6622b373db692e7ec9b96404c22d125d3dd3ff0840ba
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [17]:
from langdetect import detect
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Charger le modèle multilingue
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unknown"

def summarize_multilingual(text):
    lang = detect_lang(text)
    if lang == "ar":
        tokenizer.src_lang = "ar_AR"
        encoded = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        generated = model.generate(encoded["input_ids"], max_length=50, min_length=15)
        summary = tokenizer.decode(generated[0], skip_special_tokens=True)
    elif lang in ["fr", "en"]:
        # Repasser au modèle français/anglais (si déjà chargé)
        summary = summarizer(text, max_length=40, min_length=15, do_sample=False)[0]["summary_text"]
    else:
        summary = ""
    return summary


In [18]:
from tqdm import tqdm
tqdm.pandas()  # pour barre de progression

df["resume_annonce"] = df["texte_clean"].progress_apply(summarize_multilingual)


100%|██████████| 125/125 [08:51<00:00,  4.25s/it]


In [19]:
df["resume_annonce"]

0      Land for construction: 340 square meters in a ...
1      Land for sale: 1,000 square meters in certifie...
2      Land for sale in a certified Sierra Leone stat...
3       Aussi zone villas a v ter 322m2 facade 16 5m ...
4       Zone acceptable pas de zone populaire a raoue...
                             ...                        
120     a vendre terrain sis a mnihla clôture dans un...
121     a louer appartement lac 2 près tunisia mall r...
122     a louer de suite appartement ennasr 2 dans un...
123     deux villas jumelée 2 duplex sur terrain de 5...
124     564m2 facade 23 5m sur 25m cloturé constructi...
Name: resume_annonce, Length: 125, dtype: object

In [20]:
# Étape 3 : Tags NLP simples
def detect_proximite_plage(texte):
    return any(keyword in texte for keyword in ["plage", "bord de mer", "vue mer", "mer", "gammarth", "el haouaria"])

def detect_constructible(texte):
    return any(keyword in texte for keyword in ["constructible", "titre bleu", "viabilisé", "titre foncier", "lotissement approuvé"])

def detect_meuble(texte):
    return any(keyword in texte for keyword in ["meublé", "meublée", "ameublement", "entièrement équipé"])

df["proximite_plage"] = df["texte_clean"].apply(detect_proximite_plage)
df["constructible"] = df["texte_clean"].apply(detect_constructible)
df["meuble"] = df["texte_clean"].apply(detect_meuble)


In [21]:
import re

def extract_telephone(texte):
    # Extrait tous les numéros au format tunisien
    matches = re.findall(r"(?:\+216\s?|\b)(\d{8})\b", texte)
    return ", ".join(matches) if matches else None

df["telephone"] = df["texte_clean"].apply(extract_telephone)


In [22]:
df["telephone"]

0                    None
1                    None
2                    None
3                53999911
4                53999911
              ...        
120                  None
121    50030478, 98332350
122    50030478, 98332350
123              53999911
124              53999911
Name: telephone, Length: 125, dtype: object

In [26]:
import re

def nettoyer_surface(texte):
    if pd.isnull(texte):
        return texte
    return re.sub(r"\s?m²", "", texte)

df["Surface"] = df["Surface"].astype(str).apply(nettoyer_surface)
df["Surface"]

0        340
1      1 000
2      1 080
3        322
4        776
       ...  
120      370
121      135
122       73
123      509
124      564
Name: Surface, Length: 125, dtype: object

In [27]:
def extraire_categorie_principale(texte):
    if pd.isnull(texte):
        return texte
    return texte.split(">")[0].strip()

df["Catégorie"] = df["Catégorie"].apply(extraire_categorie_principale)
df["Catégorie"]

0      Offres
1      Offres
2      Offres
3      Offres
4      Offres
        ...  
120    Offres
121    Offres
122    Offres
123    Offres
124    Offres
Name: Catégorie, Length: 125, dtype: object

In [28]:
# Étape 4 : Export du fichier enrichi
df.to_excel("/content/drive/MyDrive/TP Python/annonces_enrichies.xlsx", index=False)
print("✅ Fichier enrichi sauvegardé sous 'annonces_enrichies.xlsx'")


✅ Fichier enrichi sauvegardé sous 'annonces_enrichies.xlsx'


***Phase 2 : Statistiques avec Pandas***

Manipulation et traitement des données pour générer les statistiques


In [59]:
import pandas as pd
import matplotlib.pyplot as plt

#Chargement et prétraitement
df = pd.read_excel("/content/drive/MyDrive/TP Python/annonces_enrichies.xlsx")
df.columns = df.columns.str.strip()
df

Lien Catégorie Surface  \
0    http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     340   
1    http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres   1 000   
2    http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres   1 080   
3    http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     322   
4    http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     776   
..                                                 ...       ...     ...   
120  http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     370   
121  http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     135   
122  http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres      73   
123  http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     509   
124  http://www.tunisie-annonce.com/DetailsAnnonceI...    Offres     564   

                                         Texte complet  \
0    ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...   
1    ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...   
2    ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...   
3    je  dix  bien  constructible  aussi  zone  vil...   
4    zone  acceptable  ((   pas  de  zone  populair...   
..                                                 ...   
120  a vendre terrain sis a mnihla clôture  dans un...   
121  a louer appartement  lac 2  près tunisia mall ...   
122  a  louer de suite   appartement  ennasr 2   da...   
123  possible  deux villas  jumelée  ((  2  duplex ...   
124  pour  habitation  ou investissement  des  appa...   

                     Texte annonce  Insérée le Modifiée le     Pays    Ville  \
0               Terrain nu manouba  10/12/2024  22/03/2025  Tunisie  Manouba   
1              Terrain nu chorfech  20/04/2024  22/03/2025  Tunisie   Ariana   
2                Jabbes terrain nu  18/10/2022  22/03/2025  Tunisie   Ariana   
3    Zone  villas  ter 321m2  cons  22/07/2024  22/03/2025  Tunisie   Ariana   
4        Aux  choix   776m2  507m2  22/07/2024  22/03/2025  Tunisie   Ariana   
..                             ...         ...         ...      ...      ...   
120                 Terrain mnihla  11/02/2025  22/03/2025  Tunisie   Ariana   
121           Appartement richemen  14/01/2025  22/03/2025  Tunisie    Tunis   
122           Appartement  cite  e  14/10/2024  22/03/2025  Tunisie   Ariana   
123  Pour 2 duplex  tr 510m2  angl  08/03/2025  22/03/2025  Tunisie    Tunis   
124   562m2 a 5mn de esprit  aussi  22/11/2024  22/03/2025  Tunisie   Ariana   

              Région    Nature         Type Prix (TND)  \
0        La Mannouba   Terrain   Terrain nu    221 000   
1           Chorfech   Terrain   Terrain nu    150 000   
2             Jabbes   Terrain   Terrain nu    248 400   
3         Chotrana 1   Terrain   Terrain nu    240 000   
4             Ariana   Terrain   Terrain nu    271 000   
..               ...       ...          ...        ...   
120           Mnihla     Vente     Surfaces    300 000   
121  Jardins de Cart  Location  App. 5 pièc      3 600   
122  Cite El Intissa  Location  App. 4 pièc      1 200   
123       El Manar 1   Terrain   Terrain nu    500 000   
124     Cite Essaada   Terrain   Terrain nu    450 000   

                                           texte_clean  \
0    ارض صالحة للبناء مساحة 340 متر مربع فى منوبة م...   
1    ارض للبيع مساحة 1000 متر مربع فى شرفش معتمدية ...   
2    ارض للبيع فى جباس معتمدية سيدى ثابت ولاية اريا...   
3    je dix bien constructible aussi zone villas a ...   
4    zone acceptable pas de zone populaire a raoued...   
..                                                 ...   
120  a vendre terrain sis a mnihla clôture dans une...   
121  a louer appartement lac 2 près tunisia mall ri...   
122  a louer de suite appartement ennasr 2 dans une...   
123  possible deux villas jumelée 2 duplex sur terr...   
124  pour habitation ou investissement des appartem...   

                                        resume_annonce  proximite_plage  \
0    Land 

In [60]:
#Nettoyage et conversions
df["Prix (TND)"] = (
    df["Prix (TND)"]
    .astype(str)
    .str.replace(r"[^\d.]", "", regex=True)  # Supprime tout sauf chiffres et points
    .replace("", pd.NA)                     # Remplace les chaînes vides par NaN
    .astype(float)
)
df["Surface"] = pd.to_numeric(df["Surface"], errors="coerce")
df["prix_m2"] = df["Prix (TND)"] / df["Surface"]

df["Insérée le"] = pd.to_datetime(df["Insérée le"], errors="coerce")


In [61]:
df["prix_m2"]

0      650.000000
1             NaN
2             NaN
3      745.341615
4      349.226804
          ...    
120    810.810811
121     26.666667
122     16.438356
123    982.318271
124    797.872340
Name: prix_m2, Length: 125, dtype: float64

In [62]:
#KPIs (Statistiques de base)
#Prix moyen au m²
prix_m2_moyen = df["prix_m2"].mean()
prix_m2_moyen

np.float64(1267.449206004751)

In [63]:
#Répartition des types
repartition_types = df["Type"].value_counts(normalize=True) * 100
repartition_types

Type
Terrain nu     48.0
Maisons        20.0
App. 3 pièc    16.0
App. 1 pièc     4.0
Surfaces        4.0
App. 5 pièc     4.0
App. 4 pièc     4.0
Name: proportion, dtype: float64

In [64]:
#Annonces par région
annonces_region = df["Région"].value_counts()
annonces_region


Région
La Soukra          15
Ariana             10
La Mannouba        10
Borj Touil         10
Cite Touristiqu    10
Jabbes              5
Chorfech            5
Kelibia             5
Ras Tabia           5
Cite Ennasr 2       5
Chotrana 1          5
Ain Zaghouan        5
Gammart             5
Chotrana 3          5
Mnihla              5
Jardins de Cart     5
Cite El Intissa     5
El Manar 1          5
Cite Essaada        5
Name: count, dtype: int64

In [65]:
# % proches plage
pourcentage_plage = df["proximite_plage"].mean() * 100
pourcentage_plage

np.float64(20.0)

In [66]:
histogramme_prix = df["Prix (TND)"].dropna()
histogramme_prix #without nan

0      221000.0
1      150000.0
2      248400.0
3      240000.0
4      271000.0
         ...   
120    300000.0
121      3600.0
122      1200.0
123    500000.0
124    450000.0
Name: Prix (TND), Length: 125, dtype: float64

In [67]:
distribution_surface = df["Surface"].dropna()
distribution_surface

0      340.0
3      322.0
4      776.0
5      384.0
7      301.0
       ...  
120    370.0
121    135.0
122     73.0
123    509.0
124    564.0
Name: Surface, Length: 105, dtype: float64

In [68]:
stats = df[["Prix (TND)", "Surface", "prix_m2"]].describe()
stats


Prix (TND)     Surface      prix_m2
count     125.000000  105.000000   105.000000
mean   186964.000000  246.857143  1267.449206
std    180383.562279  209.900225  1914.479268
min       550.000000    1.000000     5.000000
25%      3600.000000   73.000000    26.666667
50%    160000.000000  135.000000   650.000000
75%    271000.000000  370.000000   982.318271
max    680000.000000  776.000000  7000.000000

In [69]:
biens_luxe = df.nlargest(10, "Prix (TND)")[["Ville", "Type", "Prix (TND)", "Surface", "prix_m2"]]

In [70]:
biens_luxe #sélectionne les 10 lignes avec les prix les plus élevés.



Ville        Type  Prix (TND)  Surface     prix_m2
9    Ariana  Terrain nu    680000.0      NaN         NaN
34   Ariana  Terrain nu    680000.0      NaN         NaN
59   Ariana  Terrain nu    680000.0      NaN         NaN
84   Ariana  Terrain nu    680000.0      NaN         NaN
109  Ariana  Terrain nu    680000.0      NaN         NaN
23    Tunis  Terrain nu    500000.0    509.0  982.318271
48    Tunis  Terrain nu    500000.0    509.0  982.318271
73    Tunis  Terrain nu    500000.0    509.0  982.318271
98    Tunis  Terrain nu    500000.0    509.0  982.318271
123   Tunis  Terrain nu    500000.0    509.0  982.318271

In [71]:
biens_aberrants = df[(df["Prix (TND)"] > 2_000_000) | (df["Surface"] > 1000)]
biens_aberrants #du coup pas de valeur abberantes

Empty DataFrame
Columns: [Lien, Catégorie, Surface, Texte complet, Texte annonce, Insérée le, Modifiée le, Pays, Ville, Région, Nature, Type, Prix (TND), texte_clean, resume_annonce, proximite_plage, constructible, meuble, telephone, prix_m2]
Index: []

KPIS (Profil du marché)

In [72]:
repartition_meuble = df["meuble"].value_counts(normalize=True) * 100
repartition_meuble

meuble
False    84.0
True     16.0
Name: proportion, dtype: float64

In [73]:
pourcentage_meuble = df["meuble"].mean() * 100
pourcentage_meuble


np.float64(16.0)

In [74]:
repartition_nature = df["Nature"].value_counts(normalize=True) * 100
repartition_nature

Nature
Terrain     48.0
Location    36.0
Vente       16.0
Name: proportion, dtype: float64

In [75]:
pourcentage_constructible = df["constructible"].mean() * 100

pourcentage_constructible

np.float64(28.000000000000004)

KPIS(Analyse géographique)

In [76]:
annonces_par_region = df["Région"].value_counts().reset_index()
annonces_par_region

Région  count
0         La Soukra     15
1            Ariana     10
2       La Mannouba     10
3        Borj Touil     10
4   Cite Touristiqu     10
5            Jabbes      5
6          Chorfech      5
7           Kelibia      5
8         Ras Tabia      5
9     Cite Ennasr 2      5
10       Chotrana 1      5
11     Ain Zaghouan      5
12          Gammart      5
13       Chotrana 3      5
14           Mnihla      5
15  Jardins de Cart      5
16  Cite El Intissa      5
17       El Manar 1      5
18     Cite Essaada      5

In [77]:
annonces_par_ville = df["Ville"].value_counts().head(10).reset_index()
annonces_par_ville

Ville  count
0   Ariana     75
1    Tunis     25
2   Nabeul     15
3  Manouba     10

In [78]:
plage_par_region = df[df["proximite_plage"] == 1]["Région"].value_counts().reset_index()
plage_par_region

Région  count
0  Cite Touristiqu     10
1           Ariana      5
2          Kelibia      5
3          Gammart      5

In [79]:
constructible_ratio = df.groupby("Région")["constructible"].mean().sort_values(ascending=False) * 100
constructible_ratio

Région
Ariana             100.0
Cite Touristiqu    100.0
Chotrana 1         100.0
Cite Essaada       100.0
Cite Ennasr 2      100.0
Borj Touil           0.0
Ain Zaghouan         0.0
Cite El Intissa      0.0
Chotrana 3           0.0
Chorfech             0.0
El Manar 1           0.0
Gammart              0.0
Jabbes               0.0
Jardins de Cart      0.0
Kelibia              0.0
La Mannouba          0.0
La Soukra            0.0
Mnihla               0.0
Ras Tabia            0.0
Name: constructible, dtype: float64

In [80]:
prix_moyen_par_ville = df.groupby("Ville")["Prix (TND)"].mean()
prix_moyen_par_ville

Ville
Ariana     218623.333333
Manouba    221000.000000
Nabeul     146666.666667
Tunis      102550.000000
Name: Prix (TND), dtype: float64

In [81]:
surface_moyenne_par_region = df.groupby("Région")["Surface"].mean()
surface_moyenne_par_region

Région
Ain Zaghouan         1.000000
Ariana             580.000000
Borj Touil         470.000000
Chorfech                  NaN
Chotrana 1         322.000000
Chotrana 3           1.000000
Cite El Intissa     73.000000
Cite Ennasr 2      128.000000
Cite Essaada       564.000000
Cite Touristiqu    301.000000
El Manar 1         509.000000
Gammart              1.000000
Jabbes                    NaN
Jardins de Cart    135.000000
Kelibia            100.000000
La Mannouba        340.000000
La Soukra           86.666667
Mnihla             370.000000
Ras Tabia          109.000000
Name: Surface, dtype: float64

In [82]:
annonces_par_pays = df["Pays"].value_counts()
annonces_par_pays # juste la tunisie

Pays
Tunisie    125
Name: count, dtype: int64

KPIS (Analyse temporelle)

In [83]:
# ⏱️ 1. Nombre d’annonces par mois
df["mois"] = df["Insérée le"].dt.to_period("M").astype(str)
annonces_par_mois = df["mois"].value_counts().sort_index()

# 📏 2. Évolution de la surface moyenne des biens dans le temps
surface_moyenne_temps = df.groupby("mois")["Surface"].mean()

# 💰 3. Évolution du prix moyen au m² dans le temps
prix_m2_moyen_temps = df.groupby("mois")["prix_m2"].mean()

# 🏘️ 4. Nombre de biens proches de la plage par mois
plage_par_mois = df[df["proximite_plage"] == 1]["mois"].value_counts().sort_index()


In [84]:
annonces_par_mois

mois
2021-02    10
2024-04     5
2024-10    15
2025-08     5
2025-11     5
NaT        85
Name: count, dtype: int64

In [85]:
prix_m2_moyen_temps

mois
2021-02       6.785714
2024-04    1600.000000
2024-10     577.297896
2025-08     982.318271
2025-11     810.810811
NaT        1652.141471
Name: prix_m2, dtype: float64

In [86]:
surface_moyenne_temps

mois
2021-02     95.000000
2024-04    100.000000
2024-10    327.000000
2025-08    509.000000
2025-11    370.000000
NaT        233.384615
Name: Surface, dtype: float64

In [87]:
plage_par_mois

mois
2024-04     5
2024-10     5
NaT        15
Name: count, dtype: int64

KPIS (Qualité des données)

In [88]:
pourcentage_surface_manquante = df["Surface"].isna().mean() * 100
pourcentage_prix_manquant = df["Prix (TND)"].isna().mean() * 100
pourcentage_telephone_manquant = df["telephone"].isna().mean() * 100
pourcentage_telephone_manquant

np.float64(68.0)

KPIS (Analyse textuelle)

In [89]:
texte_vide = df["Texte annonce"].fillna("").str.len() < 30

pourcentage_texte_vide = texte_vide.mean() * 100
pourcentage_texte_vide

np.float64(100.0)

Summary:
Manipulations et traitements déjà réalisés en haut:
Nettoyage des colonnes :

Conversion des dates (df["Insérée le"] = pd.to_datetime(...))

Création d'une colonne mois à partir des dates pour les analyses temporelles.

Calcul de variables dérivées :

prix_m2 = Prix (TND) / Surface (présumée déjà ajoutée)

Nettoyage / typage des colonnes booléennes (meuble, constructible, etc.)

Gestion des valeurs manquantes :

Calcul du pourcentage de téléphones manquants

Filtres comme df[df["proximite_plage"] == 1]

GroupBy et agrégations pour créer des statistiques :

Moyennes par mois

Moyennes par région

Classements (top_villes, biens_luxe, etc.)





---


**Phase 3 : Visualisation avec Plotly**
👉 Objectif : transformer les stats en jolis graphs interactifs



In [90]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [91]:

px.histogram(df, x="Prix (TND)", nbins=50, title="Répartition des prix").show()


In [92]:

px.histogram(df, x="Surface", nbins=50, title="Répartition des surfaces").show()


In [93]:
px.histogram(df, x="prix_m2", nbins=50, title="Répartition des prix au m²").show()



In [94]:
#PROFIL DU MARCHÉ
#pourcentage_meuble = df["meuble"].mean() * 100

px.pie(names=["Meublé", "Non meublé"], values=[pourcentage_meuble, 100 - pourcentage_meuble], title="Part des biens meublés").show()

pourcentage_constructible = df["constructible"].mean() * 100
px.pie(names=["Constructible", "Non constructible"], values=[pourcentage_constructible, 100 - pourcentage_constructible], title="Part des biens constructibles").show()


In [115]:
# 🏙️ ANALYSE GÉOGRAPHIQUE

# Répartition par type de bien
px.pie(df, names="Type", title="Répartition par type de bien").show()

# Nombre d'annonces par région
#annonces_par_region = df["Région"].value_counts().reset_index()

annonces_par_region.columns = ["Région", "Nombre"]
px.bar(annonces_par_region, x="Région", y="Nombre", title="Nombre d'annonces par région").show()
# Top 10 des villes les plus annoncées
#annonces_par_ville = df["Ville"].value_counts().head(10).reset_index()

annonces_par_ville.columns = ["Ville", "Nombre"]
px.bar(annonces_par_ville, x="Ville", y="Nombre", title="Top 10 des villes les plus annoncées").show()

# Proximité plage par région
#plage_par_region = df[df["proximite_plage"] == 1]["Région"].value_counts().reset_index()
plage_par_region.columns = ["Région", "Nombre"]
px.bar(plage_par_region, x="Région", y="Nombre", title="Biens proches de la plage par région").show()




In [113]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Filtrer uniquement les régions où il y a au moins un bien constructible
regions_constructibles = df[df["constructible"] == 1]["Région"].dropna()

# Générer le nuage de mots
text = " ".join(regions_constructibles)
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

# Affichage
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Régions avec des biens constructibles")
plt.show()
#Word Cloud des régions constructibles

In [96]:
import plotly.express as px
import pandas as pd

# ✅ Pourcentage des annonces avec et sans numéro de téléphone
df["téléphone_disponible"] = df["telephone"].notna()
tel_stats = df["téléphone_disponible"].value_counts(normalize=True) * 100
tel_stats = tel_stats.rename({True: "Téléphone présent", False: "Téléphone manquant"}).reset_index()
tel_stats.columns = ["Statut", "Pourcentage"]

# 📊 Graphe
fig = px.bar(
    tel_stats,
    x="Statut",
    y="Pourcentage",
    title="📵 Disponibilité des numéros de téléphone",
    text_auto=".2f",
    color="Statut",
    color_discrete_sequence=["red","green"]
)
fig.update_layout(yaxis_title="Pourcentage (%)")
fig.show()


In [117]:
px.box(df, y=["Prix (TND)"], title="Boîte à moustaches : Prix").show()

In [98]:
import plotly.express as px

# 📈 Nombre d’annonces par mois
px.bar(annonces_par_mois, title="Nombre d'annonces par mois").show()

# 📈 Surface moyenne par mois
px.line(surface_moyenne_temps, title="Surface moyenne des biens par mois").show()

# 📈 Prix moyen au m² par mois
px.line(prix_m2_moyen_temps, title="Prix moyen au m² par mois").show()

# 📈 Biens proches de la plage par mois
px.bar(plage_par_mois, title="Biens proches de la plage par mois").show()


In [99]:
px.pie(df, names="Nature", title="Répartition des annonces par nature").show()


In [100]:
prix_m2_type = df.groupby("Type")["prix_m2"].mean().sort_values(ascending=False).reset_index()
px.bar(prix_m2_type, x="Type", y="prix_m2", title="Prix moyen au m² par type de bien").show()


In [101]:
surface_moyenne_type = df.groupby("Type")["Surface"].mean().sort_values(ascending=False).reset_index()
px.bar(surface_moyenne_type, x="Type", y="Surface", title="Surface moyenne par type de bien").show()


In [102]:
prix_moyen_region = df.groupby("Région")["Prix (TND)"].mean().sort_values(ascending=False).reset_index()
px.bar(prix_moyen_region, x="Région", y="Prix (TND)", title="Prix moyen des biens par région").show()


In [103]:
taux_tel_region = df.groupby("Région")["telephone"].apply(lambda x: x.notna().mean() * 100).sort_values(ascending=False).reset_index()
px.bar(taux_tel_region, x="Région", y="telephone", title="Taux de présence de numéro de téléphone par région (%)").show()


In [104]:
import plotly.figure_factory as ff

# Sélection des colonnes numériques pertinentes
cols_corr = df[["Prix (TND)", "Surface", "prix_m2"]].dropna()
corr_matrix = cols_corr.corr()

# Création de la heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.index.tolist(),
    colorscale="Viridis",
    showscale=True,
    annotation_text=[[f"{val:.2f}" for val in row] for row in corr_matrix.values]
)
fig.update_layout(title_text="Matrice de corrélation")
fig.show()
#Heatmap de corrélation(Utile pour détecter les liens entre prix, surface, prix/m², etc.)

In [105]:
!pip install geopy
from geopy.geocoders import Nominatim
import time
# Installer et importer les librairies nécessaires

In [107]:
geolocator = Nominatim(user_agent="geoapi")
villes_uniques = df["Ville"].dropna().unique()

# Dictionnaire pour stocker les coordonnées
coords = {}

# Boucle pour géocoder chaque ville
for ville in villes_uniques:
    try:
        location = geolocator.geocode(f"{ville}, Tunisie")
        if location:
            coords[ville] = (location.latitude, location.longitude)
        else:
            coords[ville] = (None, None)
        time.sleep(1)  # éviter les requêtes trop rapides
    except:
        coords[ville] = (None, None)

# Appliquer les coordonnées au dataframe
df["latitude"] = df["Ville"].map(lambda x: coords.get(x, (None, None))[0])
df["longitude"] = df["Ville"].map(lambda x: coords.get(x, (None, None))[1])
#Géo-localiser les villes et ajouter les colonnes

In [108]:
# Créer la carte interactive
import plotly.express as px

df_geo = df.dropna(subset=["latitude", "longitude", "Surface"])

fig = px.scatter_mapbox(
    df_geo,
    lat="latitude",
    lon="longitude",
    color="Prix (TND)",
    size="Surface",
    hover_name="Ville",
    hover_data=["Type", "Surface", "prix_m2"],
    zoom=5,
    mapbox_style="open-street-map",
    title="Carte des biens immobiliers en Tunisie"
)
fig.show()


In [118]:
fig = px.sunburst(df, path=["Catégorie", "Nature", "Type"], title="Répartition hiérarchique des annonces")
fig.show()
#Sunburst des annonces par catégorie > nature > type
#Analyse hiérarchique des biens

In [112]:
fig = px.scatter(df, x="Surface", y="Prix (TND)", color="Type",
                 hover_data=["Ville", "Région"], title="Relation Surface vs Prix par Type de bien")
fig.show()
#Corrélation entre surface et prix par type
#Montre s’il y a une relation entre surface et prix.


***🖥️ Phase 4 : Intégration dans Dash***
👉 Objectif : regrouper tout dans une seule interface utilisateur

✅ Composants :

Filtres dynamiques (type, ville, etc.)

Graphiques interactifs

Zone résumé / stats globales

Possibilité de trier / explorer

In [151]:
!pip install dash
# Installer Dash et JupyterDash
!pip install jupyter-dash



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [157]:
import dash
from dash import dcc, html
#dash en local